# Load packages

In [1]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

# Load data

In [4]:
parent_folder = '/home/myp7435/downsampled_videos/1217'
label_paths = [os.path.join(parent_folder, 'labeled', 'cam_'+str(i)+'_labeled.csv') for i in range(4)]
# infer_paths = [os.path.join(parent_folder, 'experiment', 'cam_'+str(i)+'_trimmed_output.csv') for i in range(4)]
# infer_paths = [os.path.join(parent_folder, 'experiment_sam', 'cam_'+str(i)+'_output.csv') for i in range(4)]
infer_paths = [os.path.join(parent_folder, 'experiment_dlc', 'cam_'+str(i)+'_output.csv') for i in range(4)]


joints = ['Wrist', 'CMC_thumb', 'MCP_thumb', 'MCP1', 'MCP2', 'MCP3', 'MCP4',
          'IP_thumb', 'PIP1', 'PIP2', 'PIP3', 'PIP4', 'Dip1', 'Dip2', 'Dip3', 'Dip4',
          'Tip_thumb', 'Tip1', 'Tip2', 'Tip3', 'Tip4']

likelihood_cutoffs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
err_likelihood_cut = {cutoff:[] for cutoff in likelihood_cutoffs}
reject_likelihood_cut = {cutoff:0 for cutoff in likelihood_cutoffs}

for label_path, infer_path in zip(label_paths, infer_paths):
    df_label = pd.read_csv(label_path, header=[2,3], index_col=0)
    df_infer = pd.read_csv(infer_path, header=[0,1], index_col=0)
    
    indices = [int(re.findall(r'\d+', file)[-1]) for file in df_label.index]
    
    
    for i, index in enumerate(tqdm(indices)):
        for joint in joints:
            for likelihood_cutoff in likelihood_cutoffs:
                if df_infer[joint]['likelihood'][i] > likelihood_cutoff:
                    label = np.array([df_label[joint]['x'][i], df_label[joint]['y'][i]])
                    infer = np.array([df_infer[joint]['x'].iloc[index], df_infer[joint]['y'].iloc[index]])
                    dist = np.linalg.norm(label - infer, axis=0)
                    err_likelihood_cut[likelihood_cutoff].append(dist)
                else:
                    reject_likelihood_cut[likelihood_cutoff] += 1

100%|██████████| 126/126 [00:43<00:00,  2.88it/s]


# Compute test-error w/ different likelihood cutoffs

In [5]:
for likelihood_cutoff in likelihood_cutoffs:
    prop_nan = reject_likelihood_cut[likelihood_cutoff]/(len(indices)*len(joints)*len(label_paths))
    err_mean = np.nanmean(err_likelihood_cut[likelihood_cutoff])
    err_stderr = np.nanstd(err_likelihood_cut[likelihood_cutoff])/np.sum(np.isfinite(err_likelihood_cut[likelihood_cutoff]))
    print(f'Test error ({likelihood_cutoff} cutoff): {err_mean:.2f} ± {err_stderr:.5f} (pixels) | {prop_nan:.3f} rejected')

Test error (0.05 cutoff): 17.36 ± 0.00621 (pixels) | 0.043 rejected
Test error (0.1 cutoff): 17.22 ± 0.00639 (pixels) | 0.090 rejected
Test error (0.15 cutoff): 17.35 ± 0.00692 (pixels) | 0.163 rejected
Test error (0.2 cutoff): 17.53 ± 0.00740 (pixels) | 0.218 rejected
Test error (0.25 cutoff): 17.54 ± 0.00779 (pixels) | 0.261 rejected
Test error (0.3 cutoff): 17.45 ± 0.00804 (pixels) | 0.296 rejected
Test error (0.35 cutoff): 17.56 ± 0.00846 (pixels) | 0.329 rejected
Test error (0.4 cutoff): 17.63 ± 0.00886 (pixels) | 0.359 rejected
Test error (0.45 cutoff): 17.82 ± 0.00952 (pixels) | 0.397 rejected
Test error (0.5 cutoff): 17.79 ± 0.01018 (pixels) | 0.438 rejected


# Compute 3-D test-error for each keypoint

In [11]:
# Load 3-D data
labeled = pd.read_csv(os.path.join(parent_folder, 'labeled', 
                                    'output_3d_data_raw.csv'))
infered = pd.read_csv(os.path.join(parent_folder, 'experiment_sam', 
                                    'output_3d_data_lpf.csv'))
#                                     'output_3d_data_raw.csv'))
    
dists = []
for j, joint in enumerate(joints):
    
    poi1_coord = np.stack([labeled[joint+'_x'], 
                           labeled[joint+'_y'], 
                           labeled[joint+'_z']], axis=1)
    poi2_coord = np.stack([infered[joint+'_x'][indices], 
                           infered[joint+'_y'][indices], 
                           infered[joint+'_z'][indices]], axis=1)
    dist = np.linalg.norm(poi1_coord - poi2_coord, axis=1)
    err_mean = np.nanmean(dist)
    err_stderr = np.nanstd(dist)/ len(dist)
    prop_nan = np.sum(np.isnan(dist)) / len(dist)
    dists.append(dist)
    print(joint+f' : {err_mean:.2f} ± {err_stderr:.5f} (pixels) | {prop_nan:.3f} rejected')
dists = np.array(dists).reshape(-1,)
err_mean = np.nanmean(dists)
err_stderr = np.nanstd(dists)/len(dists)
prop_nan = np.sum(np.isnan(dists)) / len(dists)
print(f'Total : {err_mean:.2f} ± {err_stderr:.5f} (pixels) | {prop_nan:.3f} rejected')

Wrist : 8.95 ± 0.03097 (pixels) | 0.476 rejected
CMC_thumb : 5.07 ± 0.02134 (pixels) | 0.222 rejected
MCP_thumb : 5.60 ± 0.02691 (pixels) | 0.119 rejected
MCP1 : 5.07 ± 0.02302 (pixels) | 0.008 rejected
MCP2 : 5.10 ± 0.02041 (pixels) | 0.000 rejected
MCP3 : 6.19 ± 0.02534 (pixels) | 0.024 rejected
MCP4 : 6.99 ± 0.03158 (pixels) | 0.056 rejected
IP_thumb : 6.46 ± 0.02974 (pixels) | 0.183 rejected
PIP1 : 7.00 ± 0.03743 (pixels) | 0.040 rejected
PIP2 : 8.81 ± 0.04020 (pixels) | 0.032 rejected
PIP3 : 7.95 ± 0.03085 (pixels) | 0.127 rejected
PIP4 : 9.05 ± 0.03784 (pixels) | 0.143 rejected
Dip1 : 6.77 ± 0.03433 (pixels) | 0.294 rejected
Dip2 : 12.50 ± 0.05090 (pixels) | 0.302 rejected
Dip3 : 10.61 ± 0.03959 (pixels) | 0.175 rejected
Dip4 : 8.85 ± 0.03735 (pixels) | 0.206 rejected
Tip_thumb : 7.44 ± 0.02118 (pixels) | 0.913 rejected
Tip1 : 10.93 ± 0.04876 (pixels) | 0.690 rejected
Tip2 : 13.41 ± 0.05400 (pixels) | 0.603 rejected
Tip3 : 10.01 ± 0.04842 (pixels) | 0.460 rejected
Tip4 : 13.21 ± 

### interpolate indices

In [5]:
timestamp_paths = [os.path.join(parent_folder, 'experiment', 'cam_'+str(i)+'_logfile.txt') for i in range(4)]
max_timestamps = []
timestamps = []
for timestamp_path in timestamp_paths:
    timestamp = np.loadtxt(timestamp_path)
    timestamp_diff = np.round(timestamp[1] - timestamp[0])

    if timestamp_diff > 4:
        df = df.drop(0)
        timestamp = timestamp[1:]

    timestamp -= timestamp[0]
    timestamps.append(timestamp*30)
    max_timestamps.append(timestamp[-1]*30)
    
    max_timestamp = max(max_timestamps)

OSError: /home/myp7435/downsampled_videos/1217/experiment/cam_0_logfile.txt not found.

### Compute 3-D test-error for each keypoint

In [7]:
labeled = pd.read_csv(os.path.join(parent_folder, 'labeled', 
                                    'output_3d_data_raw.csv'))
infered = pd.read_csv(os.path.join(parent_folder, 'experiment', 
                                    'output_3d_data_lpf_full.csv'))
#                                     'output_3d_data_raw_full.csv'))
    
dists = []
L = int(max_timestamp + 1)
timestamp = timestamps[3]
t = np.arange(0, L)/30

for j, joint in enumerate(joints):
    poi1_coord = np.stack([labeled[joint+'_x'], 
                           labeled[joint+'_y'], 
                           labeled[joint+'_z']], axis=1)
    
#     poi2_x = np.interp(timestamp[indices], t, infered[joint+'_x'])
#     poi2_y = np.interp(timestamp[indices], t, infered[joint+'_y'])
#     poi2_z = np.interp(timestamp[indices], t, infered[joint+'_z'])
    
    poi2_x = infered[joint+'_x'][np.round(timestamp[indices])]
    poi2_y = infered[joint+'_y'][np.round(timestamp[indices])]
    poi2_z = infered[joint+'_z'][np.round(timestamp[indices])]
    
    poi2_coord = np.stack([poi2_x, poi2_y, poi2_z], axis=1)
    dist = np.linalg.norm(poi1_coord - poi2_coord, axis=1)
    err_mean = np.nanmean(dist)
    err_stderr = np.nanstd(dist)/np.sum(np.isfinite(dist))
    prop_nan = np.sum(np.isnan(dist)) / len(dist)
    dists.append(dist)
    print(joint+f' : {err_mean:.2f} ± {err_stderr:.5f} (mm) | {prop_nan:.3f} rejected')
dists = np.array(dists).reshape(-1,)
err_mean = np.nanmean(dists)
err_stderr = np.nanstd(dists)
prop_nan = np.sum(np.isnan(dists)) / len(dists)
print(f'Total : {err_mean:.2f} ± {err_stderr:.5f} (mm) | {prop_nan:.3f} rejected')

Wrist : 9.13 ± 0.05557 (mm) | 0.421 rejected
CMC_thumb : 5.27 ± 0.02877 (mm) | 0.222 rejected
MCP_thumb : 6.01 ± 0.03050 (mm) | 0.135 rejected
MCP1 : 6.04 ± 0.03021 (mm) | 0.000 rejected
MCP2 : 5.51 ± 0.02223 (mm) | 0.008 rejected
MCP3 : 6.17 ± 0.02303 (mm) | 0.024 rejected
MCP4 : 6.99 ± 0.02818 (mm) | 0.063 rejected
IP_thumb : 5.98 ± 0.03245 (mm) | 0.230 rejected
PIP1 : 7.47 ± 0.03667 (mm) | 0.071 rejected
PIP2 : 8.69 ± 0.03879 (mm) | 0.095 rejected
PIP3 : 8.78 ± 0.03525 (mm) | 0.127 rejected
PIP4 : 9.21 ± 0.03993 (mm) | 0.127 rejected
Dip1 : 8.48 ± 0.04899 (mm) | 0.214 rejected
Dip2 : 9.91 ± 0.23380 (mm) | 0.865 rejected
Dip3 : 11.44 ± 0.05116 (mm) | 0.167 rejected
Dip4 : 9.94 ± 0.04915 (mm) | 0.262 rejected
Tip_thumb : 8.40 ± 0.11301 (mm) | 0.810 rejected
Tip1 : 11.82 ± 0.11843 (mm) | 0.667 rejected
Tip2 : 12.30 ± 0.07327 (mm) | 0.333 rejected
Tip3 : 11.52 ± 0.09897 (mm) | 0.532 rejected
Tip4 : 13.30 ± 0.12197 (mm) | 0.690 rejected
Total : 8.16 ± 4.70290 (mm) | 0.289 rejected
